# SLTimer Demo

In this notebook we will show how `SLTimer` can be used to estimate a time delay from some example data.

## Example Data

We will use the "demo1" 4-image light curve data that comes with the `PyCS` repository. Let's download this and use `PyCS` to analyze it, before showing the same operations performed by SLTimer.

In [ ]:
import os, urllib
from __future__ import print_function

In [ ]:
webdir = 'https://raw.githubusercontent.com/COSMOGRAIL/PyCS/master/demo/demo1/data/'
rdbfile = 'trialcurves.txt'
    
url = os.path.join(webdir, rdbfile)
if not os.path.isfile(rdbfile):
    urllib.urlretrieve(url, rdbfile)

In [ ]:
!wc -l $rdbfile

Be in directory /Users/milanwilliams/PyCs
Run python
(To make sure you are in the right directory; import pycs, if there is no error message, you are in the right directory).

In the second line; make sure to edit the location of the data file based on the computer

In this first script we "import" the data, in this case from a simple text file with headers (other formats are also supported, see doc)

In [ ]:
import pycs
%matplotlib inline

In [ ]:
lcs = [
        pycs.gen.lc.rdbimport(rdbfile, 'A', 'mag_A', 'magerr_A', "Trial"),
        pycs.gen.lc.rdbimport(rdbfile, 'B', 'mag_B', 'magerr_B', "Trial"),
        pycs.gen.lc.rdbimport(rdbfile, 'C', 'mag_C', 'magerr_C', "Trial"),
        pycs.gen.lc.rdbimport(rdbfile, 'D', 'mag_D', 'magerr_D', "Trial")
]

In [ ]:
pycs.gen.mrg.colourise(lcs) # Gives each curve a different colour.

# Let's shift them by the "true" time shifts, for display purposes :
lcs[1].shifttime(-5.0)
lcs[2].shifttime(-20.0)
lcs[3].shifttime(-70.0)

# We show them :
pycs.gen.lc.display(lcs)

----
Notes:

In [ ]:
# Or if you prefer to save them into a file:
pycs.gen.lc.display(lcs, filename="fig_trialcurves.pdf")
# This function has many more options...

# We undo these shifts, as from now on we "forget" about these true delays.
for l in lcs:
        l.resetshifts()

# The main point of this script : we save the raw curves into a pkl file : ADDED THE DIRECTORY 
pycs.gen.util.writepickle(lcs, "/Users/milanwilliams/PyCs/demo/demo1/data/trialcurves.pkl")

# Normally we would stop here.
# In any further scripts, you can now import the data by reading this pickle file :
lcs = pycs.gen.util.readpickle("/Users/milanwilliams/PyCs/demo/demo1/data/trialcurves.pkl")

# ... and do something with it.
for l in lcs:
        print l.longinfo()
        
# For instance, we could export the data into a text file. This line is broken, syntax error?
for l in lcs:
        l.resetshifts()
pycs.gen.util.multilcsexport(lcs, "out_trialcurves.txt", separator="\t", verbose=True, properties=None)
# Which gives in this case the same file as the one from which you read the data in first place.